В колабе представлен стакинг с помощью усреднения, он дает 121 ошибок

Икс - это выходы обученных моделей, игрек - это классы

Задача представить решение лучше

In [252]:
%%bash
wget -q https://dl.min.io/client/mc/release/linux-amd64/mc
chmod +x mc
./mc config host add new https://gos.amai.io:443

Added `new` successfully.


In [253]:
!./mc cp --recursive new/amai-models-pub/test_task/ ./

`new/amai-models-pub/test_task/y_test_task.pkl` -> `y_test_task.pkl`
`new/amai-models-pub/test_task/x_test_task.pkl` -> `x_test_task.pkl`
Total: 0 B, Transferred: 3.67 MiB, Speed: 35.00 MiB/s


In [254]:
!pip install catboost
!pip install ipywidgets

In [255]:
!pip install catalyst

In [256]:
import pickle
with open('x_test_task.pkl', 'rb') as f:
    x = pickle.load(f)
with open('y_test_task.pkl', 'rb') as f:
    y = pickle.load(f)

In [257]:
# importing numerous libs for various types of solutions
import numpy as np
import random
import os
import copy
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.model_selection import train_test_split
import catboost
from catboost import Pool, CatBoostClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from catalyst.data.sampler import BalanceClassSampler

# seed everything for reproducibility
SEED = 666
random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [258]:
def get_metrics(y, preds):
  accuracy = accuracy_score(y, preds)
  macro_f1 = f1_score(y, preds, average="macro")

  return accuracy, macro_f1

def simple_stacking(x, y):
  num_errors_mean = sum(abs(np.mean(x, -1).round() - y))
  num_errors_median = sum(abs(np.median(x, -1).round() - y))
  num_errors_combined = (np.mean(x, -1)+np.median(x, -1))/2
  num_errors_combined = sum(abs(num_errors_combined.round()-y))
  simple_preds = ((np.mean(x, -1)+np.median(x, -1))/2).round()

  accuracy, macro_f1 = get_metrics(y, simple_preds)

  return f"Amount of errors with mean based stacking: {num_errors_mean}\n\
          Amount of errors with median based stacking: {num_errors_median}\n\
          Amount of errors with combined stacking: {num_errors_combined}\n\
          Accuracy: {accuracy} Macro F1: {macro_f1}"

print(simple_stacking(x, y))

Amount of errors with mean based stacking: 121.0
          Amount of errors with median based stacking: 120.0
          Amount of errors with combined stacking: 117.0
          Accuracy: 0.9986616334934798 Macro F1: 0.997221892898418


In [259]:
# simple stacking provides 117 errors out of  87420 samples
# which is 99.99866~ accuracy to improve upon and 0.99722 macro F1

In [260]:
# checking label balance
np.unique(y, return_counts=True)

(array([0, 1]), array([75181, 12239]))

In [261]:
# def generate_features(x):
#   mean_col = np.mean(x, -1, keepdims=True)
#   median_col = np.median(x, -1, keepdims=True)
#   combined_col = ((np.mean(x, -1, keepdims=True)+np.median(x, -1, keepdims=True))/2)
#   x = np.append(x, mean_col, 1)
#   x = np.append(x, median_col, 1)
#   x = np.append(x, combined_col, 1)

#   return x

# x = generate_features(x)

In [262]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.5, random_state=SEED)

In [263]:
clf = SVC(random_state=SEED)
clf.fit(x_train, y_train)

preds = clf.predict(x_valid)
x_valid.shape[0]-np.count_nonzero(preds==y_valid)

51

In [264]:
accuracy, macro_f1 = get_metrics(y_valid, preds)
print(f"Accuracy: {accuracy} Macro F1: {macro_f1}")

Accuracy: 0.9988332189430337 Macro F1: 0.9975976520402126


In [265]:
print(confusion_matrix(y_valid, preds))

[[37503    31]
 [   20  6156]]


In [266]:
# marginal increase in accuracy/f1

In [267]:
# hyperparameters
params = {
    "iterations" : 2000,
    #"auto_class_weights" : "Balanced",
    # heavily favors minority class with auto_class_weights
    "bootstrap_type" : "Bernoulli",
    "subsample" : 0.5,
    "depth" : 1, #7
    #"task_type" : "GPU",
    "learning_rate" : 0.005,
    #"l2_leaf_reg" : 0,
    "loss_function" : "Logloss",
    "eval_metric" : "TotalF1",
    "random_seed" : SEED,
    "verbose" : True,
    "metric_period" : 100
}

In [268]:
train_data = Pool(data=x_train,
                  label=y_train,                
                )

valid_data = Pool(data=x_valid,
                  label=y_valid,
                )

model = CatBoostClassifier(**params)
model.fit(train_data, eval_set = valid_data, plot = False)

0:	learn: 0.9961175	test: 0.9969262	best: 0.9969262 (0)	total: 17.6ms	remaining: 35.1s
100:	learn: 0.9985117	test: 0.9986961	best: 0.9986961 (100)	total: 1.07s	remaining: 20.2s
200:	learn: 0.9985349	test: 0.9987650	best: 0.9987650 (200)	total: 2.04s	remaining: 18.3s
300:	learn: 0.9986268	test: 0.9988339	best: 0.9988339 (300)	total: 2.99s	remaining: 16.9s
400:	learn: 0.9986727	test: 0.9988340	best: 0.9988340 (400)	total: 3.9s	remaining: 15.5s
500:	learn: 0.9986499	test: 0.9988340	best: 0.9988340 (400)	total: 4.8s	remaining: 14.4s
600:	learn: 0.9986270	test: 0.9988111	best: 0.9988340 (400)	total: 5.71s	remaining: 13.3s
700:	learn: 0.9986270	test: 0.9988111	best: 0.9988340 (400)	total: 6.57s	remaining: 12.2s
800:	learn: 0.9985814	test: 0.9988112	best: 0.9988340 (400)	total: 7.44s	remaining: 11.1s
900:	learn: 0.9985814	test: 0.9988341	best: 0.9988341 (900)	total: 8.36s	remaining: 10.2s
1000:	learn: 0.9985814	test: 0.9988570	best: 0.9988570 (1000)	total: 9.22s	remaining: 9.2s
1100:	learn: 0

In [269]:
cv_params = model.get_params()
cv_data = catboost.cv(
                  Pool(x, y),
                  params,
                  fold_count=5,
                  )

0:	learn: 0.9970972	test: 0.9970569	best: 0.9970569 (0)
100:	learn: 0.9984664	test: 0.9984434	best: 0.9984434 (100)
200:	learn: 0.9985612	test: 0.9985582	best: 0.9985582 (200)	total: 16.8s	remaining: 2m 30s
300:	learn: 0.9986329	test: 0.9985927	best: 0.9985927 (300)
400:	learn: 0.9986645	test: 0.9986042	best: 0.9986042 (400)
500:	learn: 0.9986674	test: 0.9986158	best: 0.9986158 (500)
600:	learn: 0.9986674	test: 0.9986158	best: 0.9986158 (500)
700:	learn: 0.9986703	test: 0.9985929	best: 0.9986158 (500)
800:	learn: 0.9986646	test: 0.9986044	best: 0.9986158 (500)
900:	learn: 0.9986646	test: 0.9986159	best: 0.9986159 (900)
1000:	learn: 0.9986703	test: 0.9986159	best: 0.9986159 (900)
1100:	learn: 0.9986789	test: 0.9986159	best: 0.9986159 (900)
1200:	learn: 0.9986818	test: 0.9986159	best: 0.9986159 (900)
1300:	learn: 0.9986818	test: 0.9986159	best: 0.9986159 (900)	total: 1m 35s	remaining: 51.2s
1400:	learn: 0.9986760	test: 0.9986388	best: 0.9986388 (1400)
1500:	learn: 0.9986846	test: 0.99863

In [270]:
print(f"Best validation total F1 score: {np.max(cv_data['test-TotalF1-mean'])}±\
    {cv_data['test-TotalF1-std'][np.argmax(cv_data['test-TotalF1-mean'])]}\
    on step {np.argmax(cv_data['test-TotalF1-mean'])}")

Best validation total F1 score: 0.9986387729697727±    0.0003170138462593982    on step 14


In [271]:
preds = model.predict(x_valid)
accuracy, macro_f1 = get_metrics(y_valid, preds)
print(f"Accuracy: {accuracy} Macro F1: {macro_f1}")

Accuracy: 0.9988560970029742 Macro F1: 0.9976468233228188


In [272]:
print(confusion_matrix(y_valid, preds))

[[37497    37]
 [   13  6163]]


In [273]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    logging.info(f'Running on {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')

# hyperparameters
batch_size = 64
learning_rate = 0.00005
epochs = 20

class Net(nn.Module):
    def __init__(self, input_size=x.shape[1], output_size=2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, output_size) 

        self.dropout = nn.Dropout(0.05)   
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))
        x = self.fc4(x)

        return x

net = Net().to(device)

In [274]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.data = torch.from_numpy(x)
        self.labels = torch.from_numpy(y)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):        
        sample, label = self.data[index], self.labels[index]

        return sample, label

train_dataset, valid_dataset = Dataset(x_train, y_train), Dataset(x_valid, y_valid)

In [275]:
train_dataloader = DataLoader(
            train_dataset,
            sampler=BalanceClassSampler(y_train, mode = "upsampling"),
            #sampler = RandomSampler(train_dataset),
            batch_size = batch_size)

valid_dataloader = DataLoader(
            valid_dataset,
            sampler = SequentialSampler(valid_dataset),
            batch_size = batch_size)

In [276]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr = learning_rate, weight_decay = 0.999, betas = (0.9, 0.999))

def training_loop(epochs = epochs, net = net):
    best_model_wts = copy.deepcopy(net.state_dict())
    best_loss = 100
    
    for epoch in (range(epochs)):
        print(f'Epoch {epoch+1}')
        train_losses, train_accuracies = train(net)        
        val_losses, val_accuracies = validate(net)        
        print(f'Training accuracy:   {sum(train_accuracies)/len(train_accuracies)} | Training loss: {sum(train_losses)/len(train_losses)}')
        print(f'Validation accuracy: {sum(val_accuracies)/len(val_accuracies)} | Validation loss: {sum(val_losses)/len(val_losses)}')
        
        epoch_val_loss = sum(val_losses)/len(val_losses)
        
        if best_loss > epoch_val_loss:    
            best_loss = epoch_val_loss
            best_model_wts = copy.deepcopy(net.state_dict())
            torch.save(net.state_dict(), 'best.pth')
            print('saving with loss of {}'.format(epoch_val_loss), 'improved over previous {}'.format(best_loss))

In [277]:
def fwd_pass(X, y, step, train = False):           
    outputs = net(X)
    matches = [torch.argmax(i) == j for i, j in zip(outputs,y)]        
    acc = matches.count(True)/len(matches)
    loss = loss_function(outputs, y)
    if train:
        loss.backward()        
        optimizer.step()        
        optimizer.zero_grad()
    return acc, loss

def train(net):
    net.train()
    train_losses = []
    train_accuracies = []
    print("Training")
    for step, batch in (enumerate(train_dataloader)):
        inputs = batch[0].to(device).float()
        labels = batch[1].to(device).long()    
        acc, loss = fwd_pass(inputs, labels, step, train = True)
        if step > 0 and step % 50 == 0:            
            print(f"Step {step} of {len(train_dataloader)}, Accuracy: {sum(train_accuracies)/len(train_accuracies)}, Loss: {sum(train_losses)/len(train_losses)}")
               
        train_losses.append(loss)
        train_accuracies.append(acc)
    return train_losses, train_accuracies               
                
def validate(net):    
    net.eval()
    val_losses = []
    val_accuracies = []
    print("Validation")
    for step, batch in enumerate(valid_dataloader):            
        valid_inputs = batch[0].to(device).float()
        valid_labels = batch[1].to(device).long()
    with torch.no_grad():
        val_acc, val_loss =  fwd_pass(valid_inputs, valid_labels, step, train = False)
        
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)        
    return val_losses, val_accuracies

In [278]:
training_loop(50)

Epoch 1
Training
Step 50 of 1177, Accuracy: 0.843125, Loss: 0.6607733964920044
Step 100 of 1177, Accuracy: 0.92015625, Loss: 0.6074382066726685
Step 150 of 1177, Accuracy: 0.9454166666666667, Loss: 0.5413172841072083
Step 200 of 1177, Accuracy: 0.9584375, Loss: 0.4698528051376343
Step 250 of 1177, Accuracy: 0.966125, Loss: 0.4043361246585846
Step 300 of 1177, Accuracy: 0.9711979166666667, Loss: 0.35051238536834717
Step 350 of 1177, Accuracy: 0.9748214285714286, Loss: 0.3074113726615906
Step 400 of 1177, Accuracy: 0.9776171875, Loss: 0.2731604278087616
Step 450 of 1177, Accuracy: 0.9798263888888888, Loss: 0.24559831619262695
Step 500 of 1177, Accuracy: 0.9816875, Loss: 0.22278475761413574
Step 550 of 1177, Accuracy: 0.9830113636363637, Loss: 0.20431318879127502
Step 600 of 1177, Accuracy: 0.9842708333333333, Loss: 0.18830394744873047
Step 650 of 1177, Accuracy: 0.9852163461538461, Loss: 0.1750347763299942
Step 700 of 1177, Accuracy: 0.9860267857142857, Loss: 0.1634148508310318
Step 750 

In [279]:
net.eval()
preds = net(torch.from_numpy(x_valid).to(device))
preds = preds.argmax(1)

In [280]:
accuracy, macro_f1 = get_metrics(y_valid, preds)
print(f"Accuracy: {accuracy} Macro F1: {macro_f1}")

Accuracy: 0.9982841455044612 Macro F1: 0.9964785468190034


In [281]:
print(confusion_matrix(y_valid, preds))

[[37467    67]
 [    8  6168]]
